# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 21st 2025

**Student Name**: Luis Antonio Pelayo Sierra

**Professor**: Pablo Camarillo Ramirez

In [14]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://a6d68e2ef18e:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Schema

In [16]:
from pcamarillor.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

# Load CSV

In [17]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/Airline/")

df_airlines.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

# Drop unnecessary columns. Count how many null values the dataset has before/after the cleaning process.

In [18]:
from pyspark.sql.functions import trim, col, count, isnull, when
print(f"number of records before cleaning: {df_airlines.count()}")
# Get number of null values for each column before cleaning 
df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

# Perform data cleaning with trim (column by column)
airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())

# Simply using dropna()
airlines_clean_v2 = df_airlines.dropna()

print(f"number of records after cleaning with trim: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

print(f"number of records after cleaning with dropna: {airlines_clean_v2.count()}")
airlines_clean_v2.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with trim: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with dropna: 300153


[Stage 28:=============================>                            (1 + 1) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



# Normalize categorical values: map “zero” → 0, “one” → 1, etc. in stops.

In [20]:
from pyspark.sql.functions import when, lit
# Perform the 1st transformation 
airline_t1 = airlines_clean_v2.withColumn("stops",
                                                when(airlines_clean_v2.stops == "zero", lit(0))
                                                .when(airlines_clean_v2.stops == "one", lit(1))
                                                .when(airlines_clean_v2.stops == "two", lit(2))
                                                .otherwise(col("stops")))
airline_t1.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|    0|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning|    0|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning|    0|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

# Create a new column called route: “Delhi → Mumbai” from source_city and destination_city.

In [22]:
from pyspark.sql.functions import concat_ws
df_routes= airline_t1.withColumn("route",
                                    concat_ws( " → ",col("source_city"), col("destination_city")))
df_routes.show(5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+--------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|         route|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+--------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|    0|        Night|          Mumbai|Economy|    2.17|        1| 5953|Delhi → Mumbai|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5953|Delhi → Mumbai|
|    2| AirAsia| I5-764|      Delhi| Early_Morning|    0|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|Delhi → Mumbai|
|    3| Vistara| UK-995|      Delhi|       Morning|    0|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|Delhi → Mumbai|
|    4| Vistara| UK-963|      Delhi|     

# Transform departure_time and arrival_time to numerical categories (Morning, Afternoon, etc.), then encode as numbers (0=Early_Morning, 1=Morning, etc.).

In [ ]:
df_arrivals = df_routes.withColumn("arrival_time",
                                    when(df_routes.arrival_time == "Early_Morning", lit(0))
                                    .when(df_routes.arrival_time == "Morning", lit(1))
                                    .when(df_routes.arrival_time == "Evening", lit(2))
                                    .when(df_routes.arrival_time == "Afternoon", lit(3))
                                    .when(df_routes.arrival_time == "Night", lit(4)))
df_departures = df_arrivals.withColumn("departure_time",
                                        when(df_arrivals.departure_time == "Early_Morning", lit(0))
                                        .when(df_arrivals.departure_time == "Morning", lit(1))
                                        .when(df_arrivals.departure_time == "Evening", lit(2))
                                        .when(df_arrivals.departure_time == "Afternoon", lit(3))
                                        .when(df_arrivals.departure_time == "Night", lit(4)))
df_departures.show(n=5)

+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+--------------+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|         route|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+--------------+
|    0|SpiceJet|SG-8709|      Delhi|             2|    0|           4|          Mumbai|Economy|    2.17|        1| 5953|Delhi → Mumbai|
|    1|SpiceJet|SG-8157|      Delhi|             0|    0|           1|          Mumbai|Economy|    2.33|        1| 5953|Delhi → Mumbai|
|    2| AirAsia| I5-764|      Delhi|             0|    0|           0|          Mumbai|Economy|    2.17|        1| 5956|Delhi → Mumbai|
|    3| Vistara| UK-995|      Delhi|             1|    0|           3|          Mumbai|Economy|    2.25|        1| 5955|Delhi → Mumbai|
|    4| Vistara| UK-963|      Delhi|            

# Add a new column is_expensive: when(price > 6000, True).otherwise(False).

In [24]:
df_is_expensive= df_departures.withColumn("is_expensive",
                                            when(col("price")>6000, True).otherwise(False))
df_is_expensive.show(n=5)

+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+--------------+------------+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|         route|is_expensive|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+--------------+------------+
|    0|SpiceJet|SG-8709|      Delhi|             2|    0|           4|          Mumbai|Economy|    2.17|        1| 5953|Delhi → Mumbai|       false|
|    1|SpiceJet|SG-8157|      Delhi|             0|    0|           1|          Mumbai|Economy|    2.33|        1| 5953|Delhi → Mumbai|       false|
|    2| AirAsia| I5-764|      Delhi|             0|    0|           0|          Mumbai|Economy|    2.17|        1| 5956|Delhi → Mumbai|       false|
|    3| Vistara| UK-995|      Delhi|             1|    0|           3|          Mumbai|Economy|    2.25|  

# Get the average price per airline.


In [25]:
from pyspark.sql.functions import avg, min, max, count
df_is_expensive.groupBy("airline").agg(avg("price").alias("avg_price")).show(n=5)

[Stage 35:=============================>                            (1 + 1) / 2]

+---------+------------------+
|  airline|         avg_price|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
+---------+------------------+
only showing top 5 rows


# Average duration per route.


In [26]:
df_is_expensive.groupBy("route").agg(avg("duration").alias("avg_duration")).show(n=5)

[Stage 38:=============================>                            (1 + 1) / 2]

+--------------------+------------------+
|               route|      avg_duration|
+--------------------+------------------+
|Hyderabad → Banga...| 12.09331678643705|
|    Mumbai → Kolkata|12.836848115489666|
|    Mumbai → Chennai|12.665900287564627|
|  Mumbai → Hyderabad|13.263310412247066|
|  Mumbai → Bangalore|11.612022516178817|
+--------------------+------------------+
only showing top 5 rows


# Minimum and maximum price per airline.

In [27]:
df_is_expensive.groupBy("airline").agg(min("price").alias("min_price"),
                                        max("price").alias("max_price")).show(n=5)

[Stage 41:=============================>                            (1 + 1) / 2]

+---------+---------+---------+
|  airline|min_price|max_price|
+---------+---------+---------+
|   Indigo|     1105|    31952|
| SpiceJet|     1106|    34158|
|Air_India|     1526|    90970|
|  AirAsia|     1105|    31917|
| GO_FIRST|     1105|    32803|
+---------+---------+---------+
only showing top 5 rows


# Count flights by departure_time category.


In [28]:
df_is_expensive.groupBy("departure_time").agg(count("*").alias("flight_count")).show(n=5) 

[Stage 44:=============================>                            (1 + 1) / 2]

+--------------+------------+
|departure_time|flight_count|
+--------------+------------+
|          NULL|        1306|
|             1|       71146|
|             3|       47794|
|             4|       48015|
|             2|       65102|
+--------------+------------+
only showing top 5 rows


In [ ]:
sc.stop()